In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.linear_model import Lasso

In [2]:
# Define parameters for LASSO
alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10]
#max_iter = [500, 1000, 10000, 100000]
selection = ['cyclic', 'random']
fit_intercept = [True, False]
tol = [0.0001, 0.001, 0.01, 0.1, 1, 10]
warm_start = [True, False]
#positive = [True, False]
#random_state = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
copy_X = [True, False]

# Create the random grid
random_grid = {'alpha': alpha,
               #'max_iter': max_iter,
               'selection': selection,
               'fit_intercept': fit_intercept,
               'tol': tol,
               'warm_start': warm_start,
               #'positive': positive,
               #'random_state': random_state,
               'copy_X': copy_X}

In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

scorer_mase= make_scorer(mean_absolute_scaled_error, greater_is_better=False)


In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_15min_dataset.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date']) #,format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 


In [5]:
#Feature engineering
# Create a new column for the time
data['time'] = [x for x in range(0, len(data))]

# Create the lagged features
lags = 96 + 1 # 24 hours + 1

# Create the lagged features
for i in range(1, lags):
    data['lag' + str(i)] = data['count'].shift(i)

# Drop the first three rows
data = data.dropna().reset_index(drop=True)

# Define the feature columns and the target column
feature_cols = [ f'lag{i}' for i in range(1, lags)]
target_col = 'y'

# Drop nan values
data = data.dropna()

# Rename column count to y
data = data.rename(columns={'count': 'y'})

In [6]:
# Set y to the last column
cols = list(data.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('y')) #Remove y from list
data = data[cols+['y']] #Create new dataframe with columns in the order you want

# drop the date column
train_data = np.delete(data, 0, 1) 

# Split the data into X and y
X_train, y_train = train_data[:, :-1], train_data[:, -1]


In [7]:
#vgl: https://lightrun.com/answers/scikit-learn-scikit-learn-grid_search-feeding-parameters-to-scorer-functions

# X and y to pandas dataframe
X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)


In [8]:
# Set Cross Validation to 5 iterations
tscv = TimeSeriesSplit(n_splits=5)

In [9]:
# Instantiate model
model = Lasso()

search = RandomizedSearchCV(estimator = model, 
                               param_distributions = random_grid, 
                               n_iter = 25, 
                               cv = tscv,
                               refit=True, 
                               verbose=3, 
                               random_state=42, 
                               n_jobs = -1, 
                               scoring=scorer_mase, #make_scorer(scorer_mase, greater_is_better=True), #'neg_root_mean_squared_error', #
                               error_score=np.nan)


In [10]:
# Fit the random search model
search.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[CV 1/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-0.848 total time=   0.1s
[CV 2/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-0.867 total time=   0.1s
[CV 3/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-0.871 total time=   0.2s
[CV 4/5] END alpha=0.001, copy_X=True, fit_intercept=True, selection=random, tol=10, warm_start=True;, score=-0.941 total time=   0.1s
[CV 4/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-0.855 total time=   0.4s
[CV 5/5] END alpha=0.001, copy_X=True, fit_intercept=True, selection=random, tol=10, warm_start=True;, score=-0.907 total time=   0.1s
[CV 1/5] END alpha=0.001, copy_X=True, fit_intercept=True, selection=random, tol=10, warm_start=True;, score=-0.917 total time=   0.0s
[CV 1/5] END alpha=0.1, copy_X=False, fit_intercept

RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                   estimator=Lasso(), n_iter=25, n_jobs=-1,
                   param_distributions={'alpha': [0.0001, 0.001, 0.01, 0.1, 1,
                                                  10],
                                        'copy_X': [True, False],
                                        'fit_intercept': [True, False],
                                        'selection': ['cyclic', 'random'],
                                        'tol': [0.0001, 0.001, 0.01, 0.1, 1,
                                                10],
                                        'warm_start': [True, False]},
                   random_state=42,
                   scoring=make_scorer(mean_absolute_scaled_error, greater_is_better=False),
                   verbose=3)

In [11]:
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)
print('Best Model: %s' % search.best_estimator_)
print('Best Index: %s' % search.best_index_)
print('CV Results: %s' % search.cv_results_)
print('Refit Time: %s' % search.refit_time_)
print('Scorer: %s' % search.scorer_)

Best Score: -0.8539785742853665
Best Hyperparameters: {'warm_start': False, 'tol': 0.1, 'selection': 'random', 'fit_intercept': True, 'copy_X': False, 'alpha': 0.01}
Best Model: Lasso(alpha=0.01, copy_X=False, selection='random', tol=0.1)
Best Index: 6
CV Results: {'mean_fit_time': array([0.21853147, 0.07445059, 0.05411744, 0.05039287, 0.05546699,
       0.06305199, 0.09970694, 2.14028101, 0.08330326, 0.05867214,
       0.1086607 , 0.07422447, 0.18687415, 0.05081439, 0.05115724,
       0.04946642, 1.17679601, 0.46428366, 0.05733461, 0.05076027,
       0.07449079, 0.22399926, 0.05189471, 0.18225522, 0.04539127]), 'std_fit_time': array([0.13257635, 0.03923806, 0.01492647, 0.02084656, 0.02271875,
       0.02644356, 0.04352251, 1.07786663, 0.01315194, 0.02791755,
       0.06097514, 0.03132301, 0.09004556, 0.01839595, 0.02084601,
       0.01956023, 0.6261951 , 0.25315857, 0.02258446, 0.02037679,
       0.03268023, 0.13266095, 0.01975338, 0.0890066 , 0.01590792]), 'mean_score_time': array([0